In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Đọc dữ liệu
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

In [3]:
# Tiền xử lý dữ liệu
def preprocess_data(df):
    # Mã hóa nhãn cảm xúc
    label_encoder = LabelEncoder()
    encoded_emotions = label_encoder.fit_transform(df['Emotion'])
    
    # Tokenize văn bản
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df['Text'])
    sequences = tokenizer.texts_to_sequences(df['Text'])
    
    # Pad sequences
    max_length = 100
    padded_sequences = pad_sequences(sequences, maxlen=max_length)
    
    return padded_sequences, encoded_emotions, label_encoder, tokenizer

In [4]:
# Xây dựng mô hình
def build_model(vocab_size, num_classes):
    model = Sequential([
        Embedding(vocab_size, 100, input_length=100),
        LSTM(128, return_sequences=True),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [5]:
def main():
    # Đọc dữ liệu
    df = load_data('emotion_dataset_raw.csv')
    
    # Tiền xử lý dữ liệu
    X, y, label_encoder, tokenizer = preprocess_data(df)
    
    # Chia tập dữ liệu
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Xây dựng và huấn luyện mô hình
    vocab_size = len(tokenizer.word_index) + 1
    num_classes = len(label_encoder.classes_)
    
    model = build_model(vocab_size, num_classes)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    history = model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping]
    )
    
    # Đánh giá mô hình
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'\nĐộ chính xác trên tập kiểm tra: {accuracy*100:.2f}%')
    
    # Lưu mô hình và các thành phần cần thiết
    model.save('emotion_model.h5')
    
    # Lưu tokenizer
    with open('tokenizer.pkl', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    # Lưu label encoder
    with open('label_encoder.pkl', 'wb') as handle:
        pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    print('\nĐã lưu mô hình và các thành phần cần thiết')
    return model, tokenizer, label_encoder

if __name__ == '__main__':
    main() 

Epoch 1/10


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


696/696 ━━━━━━━━━━━━━━━━━━━━ 51s 70ms/step - accuracy: 0.3725 - loss: 1.6440 - val_accuracy: 0.5671 - val_loss: 1.2337
Epoch 2/10
696/696 ━━━━━━━━━━━━━━━━━━━━ 54s 78ms/step - accuracy: 0.6653 - loss: 0.9929 - val_accuracy: 0.6188 - val_loss: 1.1089
Epoch 3/10
696/696 ━━━━━━━━━━━━━━━━━━━━ 47s 68ms/step - accuracy: 0.8073 - loss: 0.5967 - val_accuracy: 0.6257 - val_loss: 1.1725
Epoch 4/10
696/696 ━━━━━━━━━━━━━━━━━━━━ 53s 76ms/step - accuracy: 0.8859 - loss: 0.3743 - val_accuracy: 0.6161 - val_loss: 1.3143
Epoch 5/10
696/696 ━━━━━━━━━━━━━━━━━━━━ 45s 64ms/step - accuracy: 0.9232 - loss: 0.2658 - val_accuracy: 0.6115 - val_loss: 1.4772
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.6217 - loss: 1.1015



Độ chính xác trên tập kiểm tra: 62.22%

Đã lưu mô hình và các thành phần cần thiết
